### Create yeobuckner17
Atlas combines 17-network Yeo Cerebral atlas with the 17-network Buckner Cerebellar atlas. Areas of overlap between cerebellum and cerebrum are resolved by assigning cerebellar values to the overlapping voxels.

In [1]:
import nibabel as nib
import numpy as np
import pandas as pd

In [2]:
def reorient(input_file, ori, out_file=None):
    from nipype.interfaces.image import Reorient
    from nipype import Node
    import os

    reorient_node = Node(Reorient(),name="reorient_to_{}".format(ori))
    reorient_node.inputs.in_file =  os.path.abspath(input_file)
    reorient_node.inputs.orientation=ori
    reorient_results = reorient_node.run()
    results_file = os.path.abspath(reorient_results.outputs.out_file)
    if out_file:
        out_file = os.path.abspath(out_file)
        os.system(f"mv {results_file} {out_file}")
    else:
        out_file = results_file
    
    return out_file

In [3]:
# https://github.com/ThomasYeoLab/CBIG/blob/master/stable_projects/brain_parcellation/Yeo2011_fcMRI_clustering/1000subjects_reference/Yeo_JNeurophysiol11_SplitLabels/Yeo2011_17networks_N1000.split_components.glossary.csv
label_file = "./sources/Originals/Yeo2011_17networks_N1000.split_components.glossary.csv"
yeo_df = pd.read_table(label_file,sep=",")

In [4]:
yeo_df

,Label Name,Network Name,Full Component Name
0,NONE,NaN,NaN
1,17Networks_LH_VisCent_Striate,central visual,striate cortex
2,17Networks_LH_VisCent_ExStr,central visual,extrastriate cortex
3,17Networks_LH_VisPeri_StriCal,peripheral visual,striate calcarine
4,17Networks_LH_VisPeri_ExStrInf,peripheral visual,extra-striate inferior
...,...,...,...
110,17Networks_RH_DefaultB_PFCv,default B,ventral prefrontal cortex
111,17Networks_RH_DefaultC_IPL,default C,inferior parietal lobule
112,17Networks_RH_DefaultC_Rsp,default C,retrosplenial
113,17Networks_RH_DefaultC_PHC,default C,parahippocampal cortex


In [5]:
# ftp://surfer.nmr.mgh.harvard.edu/pub/data/Yeo_JNeurophysiol11_MNI152.zip
# Yeo2011_17Networks_MNI152_FreeSurferConformed1mm_LiberalMask.nii.gz transformed to FSL LAS Space
atlas_file = "./sources/yeo17_1mm.nii.gz"
atlas_img = nib.load(atlas_file)
atlas_data = atlas_img.get_fdata()

In [ ]:
# https://github.com/ThomasYeoLab/CBIG/tree/master/stable_projects/brain_parcellation/Yeo2011_fcMRI_clustering/1000subjects_reference/Yeo_JNeurophysiol11_SplitLabels/MNI152
# $PWD/Yeo2011_17Networks_N1000.split_components.FSL_MNI152_FreeSurferConformed_1mm.nii.gz transformed to FSL LAS space
panproc flirt -in $PWD/Yeo2011_17Networks_MNI152_FreeSurferConformed1mm_LiberalMask.nii.gz  \
      -ref /opt/fsl/data/standard/MNI152_T1_1mm.nii.gz \
      -out $PWD/yeo17_1mm.nii.gz \
      -applyxfm -usesqform -interp nearestneighbour 

# also can just download 

In [6]:
# https://github.com/ThomasYeoLab/CBIG/tree/master/stable_projects/brain_parcellation/Yeo2011_fcMRI_clustering/1000subjects_reference/Yeo_JNeurophysiol11_SplitLabels/MNI152
# $PWD/Yeo2011_17Networks_N1000.split_components.FSL_MNI152_FreeSurferConformed_1mm.nii.gz transformed to FSL LAS space
split_file = "./sources/yeo17_splitlabel_1mm.nii.gz"
split_img = nib.load(split_file)
split_data = split_img.get_fdata()

In [7]:
from scipy import stats

for roi in range(1,int(np.max(atlas_data))+1):
    mask=atlas_data == roi
    split_rois=split_data[mask]
    split_rois=split_rois[split_rois != 0]
    modalscore = stats.mode(split_rois)
    labs = yeo_df.iloc[int(modalscore.mode),yeo_df.columns.get_loc("Network Name")]
    print(f"network {roi} has {modalscore.mode}which is {labs}")

network 1 has 59.0which is central visual
network 2 has 62.0which is peripheral visual
network 3 has 6.0which is somatomotor A
network 4 has 10.0which is somatomotor B
network 5 has 70.0which is dorsal attention A
network 6 has 15.0which is dorsal attention B
network 7 has 80.0which is salience / ventral attention A
network 8 has 25.0which is salience / ventral attention B
network 9 has 88.0which is limbic A
network 10 has 30.0which is limbic B
network 11 has 100.0which is control C
network 12 has 93.0which is control A
network 13 has 97.0which is control B
network 14 has 114.0which is temporal parietal
network 15 has 56.0which is default C
network 16 has 106.0which is default A
network 17 has 49.0which is default B


In [9]:
label_dict = {"VisCent": 1,
            "VisPeri":2,
            "SomMotA":3,           
            "SomMotB":4,     
            "DorsAttnA":5,     
            "DorsAttnB":6,    
            "SalVentAttnA":7,     
            "SalVentAttnB":8,
            "LimbicA":9,     
            "LimbicB":10,  
            "ContC":11,    
            "ContA":12,     
            "ContB":13,  
            "DefaultD":14,  
            "DefaultC":15,  
            "DefaultA":16,  
            "DefaultB":17  
}

Above confirmed also at https://github.com/ThomasYeoLab/CBIG/blob/master/stable_projects/brain_parcellation/Yeo2011_fcMRI_clustering/1000subjects_reference/17NetworksOrderedNames.csv

In [10]:
yeo_df

,Label Name,Network Name,Full Component Name
0,NONE,NaN,NaN
1,17Networks_LH_VisCent_Striate,central visual,striate cortex
2,17Networks_LH_VisCent_ExStr,central visual,extrastriate cortex
3,17Networks_LH_VisPeri_StriCal,peripheral visual,striate calcarine
4,17Networks_LH_VisPeri_ExStrInf,peripheral visual,extra-striate inferior
...,...,...,...
110,17Networks_RH_DefaultB_PFCv,default B,ventral prefrontal cortex
111,17Networks_RH_DefaultC_IPL,default C,inferior parietal lobule
112,17Networks_RH_DefaultC_Rsp,default C,retrosplenial
113,17Networks_RH_DefaultC_PHC,default C,parahippocampal cortex


In [13]:
yeo17_df=yeo_df.copy()
yeo17_df["net17_num"]=0
yeo17_df["net17_name"]=yeo17_df["Network Name"]

In [14]:
yeo17_df

,Label Name,Network Name,Full Component Name,net17_num,net17_name
0,NONE,NaN,NaN,0,NaN
1,17Networks_LH_VisCent_Striate,central visual,striate cortex,0,central visual
2,17Networks_LH_VisCent_ExStr,central visual,extrastriate cortex,0,central visual
3,17Networks_LH_VisPeri_StriCal,peripheral visual,striate calcarine,0,peripheral visual
4,17Networks_LH_VisPeri_ExStrInf,peripheral visual,extra-striate inferior,0,peripheral visual
...,...,...,...,...,...
110,17Networks_RH_DefaultB_PFCv,default B,ventral prefrontal cortex,0,default B
111,17Networks_RH_DefaultC_IPL,default C,inferior parietal lobule,0,default C
112,17Networks_RH_DefaultC_Rsp,default C,retrosplenial,0,default C
113,17Networks_RH_DefaultC_PHC,default C,parahippocampal cortex,0,default C


In [15]:
replacement_dict = {"central": "VisCent",
            "peripheral": "VisPeri",
            "somatomotor A": "SomMotA",           
            "somatomotor B": "SomMotB",     
            "dorsal attention A": "DorsAttnA",     
            "dorsal attention B": "DorsAttnB",    
            "ventral attention A": "SalVentAttnA",     
            "ventral attention B": "SalVentAttnB",
            "limbic A": "LimbicA",     
            "limbic B": "LimbicB",  
            "control A": "ContA",     
            "control B": "ContB",  
            "control C": "ContC",     
            "default A": "DefaultA",  
            "default B": "DefaultB",  
            "default C": "DefaultC",  
            "temporal parietal": "DefaultD"  
}

In [16]:
import re
def replace_location(value):
    if not pd.isna(value):
        value = re.sub(r'\s+', ' ', value).strip()
        for key, replacement in replacement_dict.items():
            if key.lower() in value.lower():
                return replacement
    return value

In [17]:
yeo17_df["net17_name"] = yeo17_df["net17_name"].apply(replace_location)

In [18]:
yeo17_df["net17_name"].tolist()

[nan,
 'VisCent',
 'VisCent',
 'VisPeri',
 'VisPeri',
 'VisPeri',
 'SomMotA',
 'SomMotB',
 'SomMotB',
 'SomMotB',
 'SomMotB',
 'DorsAttnA',
 'DorsAttnA',
 'DorsAttnA',
 'DorsAttnB',
 'DorsAttnB',
 'DorsAttnB',
 'DorsAttnB',
 'SalventAttnA',
 'SalventAttnA',
 'SalventAttnA',
 'SalventAttnA',
 'SalventAttnA',
 'SalventAttnB',
 'SalventAttnB',
 'SalventAttnB',
 'SalventAttnB',
 'SalventAttnB',
 'SalventAttnB',
 'LimbicA',
 'LimbicB',
 'ContA',
 'ContA',
 'ContA',
 'ContA',
 'ContA',
 'ContA',
 'ContB',
 'ContB',
 'ContB',
 'ContB',
 'ContB',
 'ContB',
 'ContC',
 'ContC',
 'DefaultA',
 'DefaultA',
 'DefaultA',
 'DefaultA',
 'DefaultB',
 'DefaultB',
 'DefaultB',
 'DefaultB',
 'DefaultB',
 'DefaultC',
 'DefaultC',
 'DefaultC',
 'DefaultD',
 'VisCent',
 'VisCent',
 'VisPeri',
 'VisPeri',
 'VisPeri',
 'SomMotA',
 'SomMotB',
 'SomMotB',
 'SomMotB',
 'SomMotB',
 'DorsAttnA',
 'DorsAttnA',
 'DorsAttnA',
 'DorsAttnB',
 'DorsAttnB',
 'DorsAttnB',
 'DorsAttnB',
 'SalventAttnA',
 'SalventAttnA',
 'Sa

In [19]:
yeo17_df["net17_num"] = yeo17_df["net17_name"].map(label_dict)

In [20]:
yeo17_df

,Label Name,Network Name,Full Component Name,net17_num,net17_name
0,NONE,NaN,NaN,NaN,NaN
1,17Networks_LH_VisCent_Striate,central visual,striate cortex,1.0,VisCent
2,17Networks_LH_VisCent_ExStr,central visual,extrastriate cortex,1.0,VisCent
3,17Networks_LH_VisPeri_StriCal,peripheral visual,striate calcarine,2.0,VisPeri
4,17Networks_LH_VisPeri_ExStrInf,peripheral visual,extra-striate inferior,2.0,VisPeri
...,...,...,...,...,...
110,17Networks_RH_DefaultB_PFCv,default B,ventral prefrontal cortex,17.0,DefaultB
111,17Networks_RH_DefaultC_IPL,default C,inferior parietal lobule,15.0,DefaultC
112,17Networks_RH_DefaultC_Rsp,default C,retrosplenial,15.0,DefaultC
113,17Networks_RH_DefaultC_PHC,default C,parahippocampal cortex,15.0,DefaultC


### Add Cerebellar atlas

In [ ]:
# ftp://surfer.nmr.mgh.harvard.edu/pub/data/Buckner_JNeurophysiol11_MNI152.zip
# Buckner2011_17Networks_MNI152_FreeSurferConformed1mm_LooseMask.nii.gz transformed to FSL LAS Space

panproc flirt -in $PWD/Buckner2011_17Networks_MNI152_FreeSurferConformed1mm_LooseMask.nii.gz \
      -ref /opt/fsl/data/standard/MNI152_T1_1mm.nii.gz \
      -out $PWD/buck17_1mm.nii.gz \
      -applyxfm -usesqform -interp nearestneighbour 

In [21]:

buck_file = "./sources/buck17_1mm.nii.gz"
buck_img = nib.load(buck_file)
buck_data = buck_img.get_fdata()

### Where do the Yeo and buckner overlap

In [22]:
mask1=atlas_data > 0
mask2=buck_data > 0
combined_mask = np.logical_and(mask1,mask2)

In [23]:
atlas_data[combined_mask]

array([ 5.,  5.,  5., ..., 12., 12., 12.])

In [24]:
stats.mode(buck_data[combined_mask])

ModeResult(mode=7.0, count=1015)

In [25]:
buck_data[combined_mask]

array([12., 12., 12., ..., 12., 12., 12.])

In [26]:
yeo_data = atlas_data + buck_data

In [27]:
yeo_data[combined_mask]=buck_data[combined_mask]
yeo_data[combined_mask]

array([12., 12., 12., ..., 12., 12., 12.])

In [28]:
np.max(yeo_data)

17.0

In [29]:
final_img = nib.Nifti1Image(yeo_data, atlas_img.affine, atlas_img.header)
nib.save(final_img, "./outputs/atlas-yeobuckner17_space-MNI152NLin6Asym_res-01_ori-LAS_dseg.nii.gz")

In [30]:
net17_labeldf = pd.DataFrame(list(label_dict.keys()),columns=["label"])
net17_labeldf["index"] = net17_labeldf["label"].map(label_dict)
net17_labeldf = net17_labeldf[["index","label"]]
net17_labeldf

,index,label
0,1,VisCent
1,2,VisPeri
2,3,SomMotA
3,4,SomMotB
4,5,DorsAttnA
5,6,DorsAttnB
6,7,SalventAttnA
7,8,SalventAttnB
8,9,LimbicA
9,10,LimbicB


In [33]:
net17_labeldf.to_csv("./outputs/atlas-yeobuckner17_dseg.tsv",sep="\t",header=True, index=False)

In [32]:
input_file = "./outputs/atlas-yeobuckner17_space-MNI152NLin6Asym_res-01_ori-LAS_dseg.nii.gz"
out_file="./outputs/atlas-yeobuckner17_space-MNI152NLin6Asym_res-01_dseg.nii.gz"
final = reorient(input_file, "RAS", out_file)

250215-15:05:47,803 nipype.workflow INFO:
	 [Node] Setting-up "reorient_to_RAS" in "/tmp/tmp78wcx1da/reorient_to_RAS".
250215-15:05:47,806 nipype.workflow INFO:
	 [Node] Executing "reorient_to_RAS" <nipype.interfaces.image.Reorient>
250215-15:05:48,40 nipype.workflow INFO:
	 [Node] Finished "reorient_to_RAS", elapsed time 0.230326s.


In [32]:
final

'/groups/ryant/PANapps/PANpipelines/PAN2025_Deployment/atlas/yeobuckner/Development/atlas-yeobuckner17_space-MNI152NLin6Asym_res-02_dseg.nii.gz'